# Lambda CSV analysis example


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Working with multiple files as one Data Frame

```bash
# EDM4EIC with root-tree
k_lambda_5x41_5000evt_001.edm4eic.root
k_lambda_5x41_5000evt_002.edm4eic.root
...

# Corresponding CSV files:
k_lambda_5x41_5000evt_001.mcdis.csv
k_lambda_5x41_5000evt_001.mcpart_lambda.csv
k_lambda_5x41_5000evt_002.mcdis.csv
k_lambda_5x41_5000evt_002.mcpart_lambda.csv
...
```

Each file such as `k_lambda_5x41_5000evt_001.edm4eic.root` is based on the processing 5000-events.
It is convenient to have files split in small chunks like this, but
analysis-wise 5k events are not statistically significant.
So to get results, we want to combine several files in one dataframe.
In general it is simple with pandas, but we have one problem.
When we have multiple CSV files from different runs or datasets, each file starts its event numbering from 0:

```
File 1: evt = [0, 1, 2, 3, 4, ...]
File 2: evt = [0, 1, 2, 3, 4, ...]  ← ID Collision!
File 3: evt = [0, 1, 2, 3, 4, ...]  ← ID Collision!
```

**Problem**: Event 0 from File 1 is completely different from Event 0 from File 2, but they have the same ID!

**Solution**: Global Unique Event IDs

We need to create globally unique event IDs across all files that we open:


In [1]:
def concat_csvs_with_unique_events(files):
    """Load and concatenate CSV files with globally unique event IDs"""
    dfs = []
    offset = 0

    for file in files:
        df = pd.read_csv(file)
        df['evt'] = df['evt'] + offset
        offset = df['evt'].max() + 1
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)
